# The role of topology in decentralized learning

In [72]:
import numpy as np
import numpy.random as rd
import scipy.optimize as optim
import matplotlib.pyplot as plt

In [57]:
def DSGD_toypbm(lr, d, n, W, T):
    """
    D-SGD algorithm for the toy-problem defined in section 3 (minimization of 1/2*||x||^2)

    Inputs :
    ------------
    lr: learning rate
    d: dimension
    n: number of agents
    W: gossip matrix (symmetric, stochastic, of size n*n)
    T: number of time steps

    Prints at each iteration the state x obtained by worker i = 0
    """

    x0 = rd.normal(0,1, size = (d))
    X = np.block([[x0]]*n)

    for _ in range(T):
        for i in range(n):
            v = rd.normal(0,1, size = (d,1))
            X[i,:] = X[i,:] - lr * v.dot(v.T).dot(X[i,:])

        X = W.dot(X)

        print(X[0,:])


In [69]:
n = 10
d = 4

# Ring topology gossip matrix

W = np.block([[1/5 * np.ones((5,5)),np.zeros((5,5))],[np.zeros((5,5)),1/5 * np.ones((5,5))]])
W[4,4] = 1/10
W[4,5] = 1/10
W[5,4] = 1/10
W[5,5] = 1/10



In [61]:
DSGD_toypbm(0.4, d, n, W, 50)

[-0.43880557 -0.36034454 -0.33203404  0.09663972]
[-0.39264238 -0.3470509  -0.33036956  0.02092921]
[-0.11527067 -0.18111727 -0.20810349 -0.03407988]
[-0.10287816 -0.12966335 -0.17646973  0.05669968]
[ 0.08083779 -0.06389137  0.05406046  0.08211345]
[ 0.0261899  -0.01944133  0.08371069  0.03620867]
[ 0.0158546  -0.01586398  0.04297026  0.01120719]
[-0.01152025 -0.01724644  0.01120347  0.00485066]
[-0.01194906 -0.01097799  0.01294022 -0.00096983]
[-0.01177521 -0.01222065  0.00968102 -0.00013002]
[-0.01224505 -0.00015576  0.0063666  -0.00068699]
[-5.89918826e-03 -3.43606703e-03 -1.96159997e-05 -2.82272989e-03]
[-0.00202708 -0.00206896 -0.00235714 -0.00056992]
[-0.00045513 -0.00095327 -0.00225292 -0.00131343]
[-0.00012829 -0.00071088 -0.00168191 -0.00036599]
[-0.00034534 -0.00051705 -0.00144787 -0.00040566]
[-0.00028104 -0.00079413 -0.00104725 -0.00044101]
[-0.00032569 -0.00069229 -0.00096668 -0.00029589]
[-4.40300280e-04 -5.20228541e-04 -7.68471470e-04  8.17639138e-05]
[-0.00033127 -0.00

In [65]:
def nW(gamma, W):
    """
    Computes the effective number of neighbours of the gossip matrix W for the decay parameter gamma
    """
    lambdas = np.linalg.eigvals(W)
    denominator = np.mean(lambdas**2 / (1 - gamma*lambdas**2))
    return 1/(1-gamma) * 1/denominator

In [68]:
# The effective number of neighbours for the alone topology (W = I) should be 1 independently of gamma
nW(.5, np.eye(10))

1.0

In [73]:
def rate_toypbm(lr, zeta, W):
    """
    Computes the convergence rate of D-SGD in the toy-problem case

    Inputs:
    ----------------
    lr: learning rate of D-SGD
    zeta: noise level (= d + 2 in the case of the toy-problem in dimension d)
    W: gossip matrix used for D-SGD

    Outputs:
    ----------------
    r: convergence rate of D-SGD
    """
    f_tosolve = lambda r : r - (1 - (1 - lr)**2 - (zeta - 1) * lr**2 / nW((1-lr)**2 / (1-r), W))
    r = optim.fsolve(f_tosolve)
    return r

In [74]:
rate_toypbm(0.5, 6, np.eye(10))

TypeError: fsolve() missing 1 required positional argument: 'x0'